# YOLOv8 for big objects

In [2]:
import os 
os.chdir('/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l')
import matplotlib.pyplot as plt
from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image
import pandas as pd
import numpy as np

In [9]:
# Training a Modified Model 
!yolo detect train model=yolov8l-big.yaml data="/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/data.yaml" workers=2 batch=12 device=0 epochs=100 patience=50 name=yolov8l_big

New https://pypi.org/project/ultralytics/8.2.67 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.48 🚀 Python-3.10.14 torch-1.13.1+cu117 CUDA:0 (Tesla T4, 14918MiB)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8l-big.yaml, data=/home/jupyter/ee_tree_counting/Data/Combined Dataset YOLO No Aug/data.yaml, epochs=100, time=None, patience=50, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=2, project=None, name=yolov8l_big, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=Fal

# Inference on Test Set

In [3]:
model = YOLO("/home/jupyter/ee_tree_counting/Models/YOLOv8 Architecture/YOLOv8l/runs/detect/yolov8l_big/weights/best.pt")  

# Creating directory to store results
test_images_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/images'
output_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/YOLOv8l_big No Aug Inference'
os.makedirs(output_dir, exist_ok=True)

# List of all test images
test_images = [os.path.join(test_images_dir, img) for img in os.listdir(test_images_dir) if img.endswith('.jpg')]
test_images.sort()

# Run inference on all test images
results = model(test_images)  

# Visualize and save the results
for i, r in enumerate(results):
    # Plot results image
    im_bgr = r.plot()  # BGR-order numpy array
    im_rgb = Image.fromarray(im_bgr[..., ::-1])  # RGB-order PIL image

    # Save results to disk
    im_rgb.save(os.path.join(output_dir, f"results_{i}.jpg"))


0: 640x640 81 trees, 27.3ms
1: 640x640 9 trees, 27.3ms
2: 640x640 43 trees, 27.3ms
3: 640x640 44 trees, 27.3ms
4: 640x640 44 trees, 27.3ms
5: 640x640 44 trees, 27.3ms
6: 640x640 64 trees, 27.3ms
7: 640x640 51 trees, 27.3ms
8: 640x640 51 trees, 27.3ms
9: 640x640 51 trees, 27.3ms
10: 640x640 52 trees, 27.3ms
11: 640x640 66 trees, 27.3ms
12: 640x640 92 trees, 27.3ms
13: 640x640 92 trees, 27.3ms
14: 640x640 92 trees, 27.3ms
15: 640x640 93 trees, 27.3ms
16: 640x640 68 trees, 27.3ms
17: 640x640 79 trees, 27.3ms
18: 640x640 142 trees, 27.3ms
19: 640x640 19 trees, 27.3ms
20: 640x640 81 trees, 27.3ms
21: 640x640 81 trees, 27.3ms
22: 640x640 81 trees, 27.3ms
23: 640x640 81 trees, 27.3ms
24: 640x640 72 trees, 27.3ms
25: 640x640 72 trees, 27.3ms
26: 640x640 73 trees, 27.3ms
27: 640x640 73 trees, 27.3ms
28: 640x640 73 trees, 27.3ms
29: 640x640 72 trees, 27.3ms
30: 640x640 72 trees, 27.3ms
31: 640x640 72 trees, 27.3ms
32: 640x640 72 trees, 27.3ms
33: 640x640 81 trees, 27.3ms
34: 640x640 97 trees, 2

# Ground Truth

In [ ]:
label_dir = '/home/jupyter/ee_tree_counting/Data/Final Tree Dataset YOLO/test/labels'

# List of labels
labels = [os.path.join(label_dir, img) for img in os.listdir(label_dir) if img.endswith('.txt')]
labels.sort()

for idx, label in enumerate(labels):
    file = pd.read_csv(label, delimiter=' ', header=None)
    print(f'Ground Truth {idx}: {len(file)} trees')

In [6]:
predictions = [
    81, 9, 43, 44, 44, 44, 64, 51, 51, 51,
    52, 66, 92, 92, 92, 93, 68, 79, 142, 19,
    81, 81, 81, 81, 72, 72, 73, 73, 73, 72,
    72, 72, 72, 81, 97, 97, 97, 97, 62, 62,
    62, 62, 77, 36, 122, 122, 122, 122, 122, 78
]

ground_truth = [
    86, 7, 48, 48, 48, 48, 64, 69, 69, 69,
    69, 105, 114, 114, 114, 114, 109, 97, 175, 20,
    72, 72, 72, 72, 89, 89, 89, 89, 89, 77,
    77, 77, 77, 148, 112, 112, 112, 112, 72, 72,
    72, 72, 74, 50, 130, 130, 130, 130, 130, 89
]

# Create a DataFrame to store the results
results_df = pd.DataFrame({
    'Image': [f'Image {i}' for i in range(len(predictions))],
    'Prediction': predictions,
    'Ground Truth': ground_truth
})

# Display the dataframe
print(results_df)

# Calculate Mean Absolute Error (MAE)
mae = (results_df['Prediction'] - results_df['Ground Truth']).abs().mean()

print(f"Mean Absolute Error (MAE): {mae:.2f}")

       Image  Prediction  Ground Truth
0    Image 0          81            86
1    Image 1           9             7
2    Image 2          43            48
3    Image 3          44            48
4    Image 4          44            48
5    Image 5          44            48
6    Image 6          64            64
7    Image 7          51            69
8    Image 8          51            69
9    Image 9          51            69
10  Image 10          52            69
11  Image 11          66           105
12  Image 12          92           114
13  Image 13          92           114
14  Image 14          92           114
15  Image 15          93           114
16  Image 16          68           109
17  Image 17          79            97
18  Image 18         142           175
19  Image 19          19            20
20  Image 20          81            72
21  Image 21          81            72
22  Image 22          81            72
23  Image 23          81            72
24  Image 24          72 